# BigQuery - BQML

Multiclass Logistic Regression with BigQuery

Make a client connection to BigQuery

In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

Use BigQuery ML to train multiclass logistic regression model:

In [2]:
%%bigquery
CREATE OR REPLACE MODEL `statmike-mlops.digits.digits_lr`
OPTIONS
  ( model_type='LOGISTIC_REG',
    auto_class_weights=TRUE,
    input_label_cols=['target']
  ) AS
SELECT * EXCEPT(splits,target_OE)
FROM `statmike-mlops.digits.digits_prepped`
WHERE splits = 'TRAIN'

""


Review the iterations from training:

In [3]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `statmike-mlops.digits.digits_lr`)
ORDER BY iteration

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.194599,0.194410,0.2,3492
1,0,1,0.141340,0.141239,0.4,5256
2,0,2,0.086843,0.087746,0.8,5619
3,0,3,0.051585,0.054067,1.6,5335
4,0,4,0.032244,0.036022,3.2,5693
5,0,5,0.021431,0.025511,6.4,5915
6,0,6,0.015505,0.021830,12.8,6028
7,0,7,0.013646,0.019434,3.2,5532
8,0,8,0.012388,0.018775,6.4,5824
9,0,9,0.010573,0.017609,12.8,5281


Review the model evaluation statistics on the Test/Train splits:

In [4]:
%%bigquery
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `statmike-mlops.digits.digits_lr`,
    (SELECT * FROM `statmike-mlops.digits.digits_prepped` WHERE SPLITS='TRAIN'))
UNION ALL
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `statmike-mlops.digits.digits_lr`,
    (SELECT * FROM `statmike-mlops.digits.digits_prepped` WHERE SPLITS='TEST'))

,SPLIT,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,TEST,0.965650,0.964342,0.967552,0.964912,0.168087,0.998317
1,TRAIN,0.978813,0.978854,0.978738,0.978739,0.118961,0.999750


Review the confusion matrix for each split:

In [5]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`
    WHERE splits = 'TRAIN')
  );

,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,146,0,0,0,0,0,0,0,0,0
1,1,0,144,0,0,0,0,0,0,1,0
2,2,0,0,141,0,0,0,0,0,0,0
3,3,0,0,1,144,0,1,0,0,3,0
4,4,0,0,0,0,145,0,0,0,1,2
5,5,0,1,0,0,0,135,1,0,0,2
6,6,0,1,0,0,1,0,142,0,0,0
7,7,0,0,0,0,0,0,0,149,0,1
8,8,0,4,1,0,0,3,0,0,139,0
9,9,0,2,0,1,0,2,0,0,2,142


In [6]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`
    WHERE splits = 'TEST')
  );

,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,32,0,0,0,0,0,0,0,0,0
1,1,0,34,0,0,0,0,0,0,2,1
2,2,0,0,36,0,0,0,0,0,0,0
3,3,0,0,0,34,0,0,0,0,0,0
4,4,0,0,0,0,31,0,0,1,1,0
5,5,0,0,0,0,0,43,0,0,0,0
6,6,0,0,0,0,0,0,37,0,0,0
7,7,0,0,0,0,1,0,0,28,0,0
8,8,0,3,0,0,0,0,1,0,23,0
9,9,0,0,0,0,0,1,0,0,0,30


Create a pandas dataframe with predictions for the full table:

In [7]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `statmike-mlops.digits.digits_lr`,(
    SELECT *
    FROM `statmike-mlops.digits.digits_prepped`)
  )

In [8]:
pred.columns

Index(['predicted_target', 'predicted_target_probs', 'p0', 'p1', 'p2', 'p3',
       'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14',
       'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24',
       'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34',
       'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44',
       'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54',
       'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target',
       'target_OE', 'SPLITS'],
      dtype='object')

In [9]:
pred[['target', 'predicted_target', 'predicted_target_probs', 'SPLITS']].head()

,target,predicted_target,predicted_target_probs,SPLITS
0,0,0,"[{'label': 0, 'prob': 0.9700539615211876}, {'l...",TRAIN
1,0,0,"[{'label': 0, 'prob': 0.9697499070706559}, {'l...",TRAIN
2,1,1,"[{'label': 1, 'prob': 0.995939544282871}, {'la...",TEST
3,9,9,"[{'label': 9, 'prob': 0.8586646313509634}, {'l...",TRAIN
4,9,9,"[{'label': 9, 'prob': 0.994500287929089}, {'la...",TRAIN


In [10]:
[sorted(x, key = lambda x: x['label']) for x in [pred.predicted_target_probs[0]]]

[[{'label': 0, 'prob': 0.9700539615211876},
  {'label': 1, 'prob': 2.518177689824362e-05},
  {'label': 2, 'prob': 0.001623011973809738},
  {'label': 3, 'prob': 0.0006080583973973493},
  {'label': 4, 'prob': 0.0025883214911664072},
  {'label': 5, 'prob': 0.001118377736220435},
  {'label': 6, 'prob': 0.017950984945240396},
  {'label': 7, 'prob': 0.0008685308769735511},
  {'label': 8, 'prob': 0.0009940653481727228},
  {'label': 9, 'prob': 0.004169505932933622}]]

# Remove Resources
- delete model `statmike-mlops.digits.digits_lr`

In [2]:
bq.delete_model('statmike-mlops.digits.digits_lr')